In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Reading data file
df = pd.read_csv('train_clas.csv')

In [3]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

df_cont = df.select_dtypes(include=numerics)
df_cat = df.select_dtypes(include = 'object')

In [4]:
df.shape

(891, 12)

In [5]:
df_cont.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

In [6]:
df_cont['Age'] = df_cont['Age'].fillna(df_cont['Age'].median())

In [7]:
df_cont.drop(['PassengerId'], axis=1, inplace=True)

In [8]:
df_cat.isnull().sum()

Name          0
Sex           0
Ticket        0
Cabin       687
Embarked      2
dtype: int64

In [9]:
df_cat['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [10]:
df_cat['Embarked'] = df_cat['Embarked'].fillna('S')

In [11]:
df_cat.drop(['Cabin', 'Name'], axis=1, inplace=True)

In [12]:
df_response = df['Survived']
df_cont = df_cont.loc[:, df_cont.columns != 'Survived']

In [13]:
#lasso with lasso cv

X_lasso = df_cont
y = df_response

from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV

cv = KFold(n_splits=10, shuffle=True, random_state=10)
model = LassoCV(cv=cv)
lassocv = model.fit(X_lasso, y)

B_lasso = lassocv.coef_
B_lasso = np.insert(B_lasso, 0, lassocv.intercept_, axis=0)

print('The Coef are')
print(B_lasso)
#print('\nlambda best is = %f' % lassocv.alpha_)

The Coef are
[ 1.02035562e+00 -1.93050483e-01 -7.30333842e-03 -3.42491473e-02
  3.00430600e-02  9.33204693e-04]


In [14]:
df_cont2 =  df_cont * lassocv.coef_
df_cont2 = df_cont2.loc[:, (df_cont2 != 0).any(axis=0)]
df_cont2

,Pclass,Age,SibSp,Parch,Fare
0,-0.579151,-0.160673,-0.034249,0.000000,0.006766
1,-0.193050,-0.277527,-0.034249,0.000000,0.066522
2,-0.579151,-0.189887,-0.000000,0.000000,0.007396
3,-0.193050,-0.255617,-0.034249,0.000000,0.049553
4,-0.579151,-0.255617,-0.000000,0.000000,0.007512
...,...,...,...,...,...
886,-0.386101,-0.197190,-0.000000,0.000000,0.012132
887,-0.193050,-0.138763,-0.000000,0.000000,0.027996
888,-0.579151,-0.204493,-0.034249,0.060086,0.021884
889,-0.193050,-0.189887,-0.000000,0.000000,0.027996


In [15]:
def outliers (dataframe):
    
    for column in dataframe:
        Q1 = dataframe[column].quantile(0.25)
        Q3 = dataframe[column].quantile(0.75)
        IQR = Q3 - Q1
        Lower_Bound = Q1 - 1.5*IQR
        Upper_Bound = Q3 + 1.5*IQR
        
        for i in np.linspace(0,len(dataframe[column])-1, len(dataframe[column])):
            if dataframe[column][i] < Lower_Bound:
                dataframe[column][i] = Q1
            elif dataframe[column][i] > Upper_Bound:
                dataframe[column][i] = Q3
                
    return dataframe

In [16]:
df_cont2 = outliers(df_cont2)

In [17]:
#one hot encoding for categorical
df_cat2 = pd.get_dummies(df_cat, dummy_na=True)

In [18]:
df_final = pd.concat([df_cont2, df_cat2, df_response], axis = 1)

In [19]:
df_final

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Sex_nan,Ticket_110152,Ticket_110413,...,Ticket_W./C. 6609,Ticket_W.E.P. 5734,Ticket_W/C 14208,Ticket_WE/P 5735,Ticket_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Survived
0,-0.579151,-0.160673,-0.034249,0.0,0.006766,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-0.193050,-0.277527,-0.034249,0.0,0.028929,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,-0.579151,-0.189887,-0.000000,0.0,0.007396,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,-0.193050,-0.255617,-0.034249,0.0,0.049553,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,-0.579151,-0.255617,-0.000000,0.0,0.007512,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-0.386101,-0.197190,-0.000000,0.0,0.012132,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
887,-0.193050,-0.138763,-0.000000,0.0,0.027996,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
888,-0.579151,-0.204493,-0.034249,0.0,0.021884,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
889,-0.193050,-0.189887,-0.000000,0.0,0.027996,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [20]:
from sklearn.decomposition import PCA

#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler

pca = PCA()

x = df_final.loc[:, df_final.columns != 'Survived']
y = df_final['Survived']

x_pca = pca.fit_transform(x)
x_pca = pd.DataFrame(x_pca)
x_pca.head()

,0,1,2,3,4,5,6,7,8,9,...,684,685,686,687,688,689,690,691,692,693
0,-0.580832,-0.188935,-0.010679,-0.116232,0.008656,-0.009090,-0.019332,-0.002889,-0.002352,-0.010705,...,3.999504e-10,2.848324e-16,5.920432e-17,2.408456e-17,4.793126e-18,-1.151978e-19,-1.282902e-19,1.970825e-19,-1.397036e-19,1.327540e-20
1,1.176686,0.753156,-0.358226,0.136653,-0.012340,0.010168,0.008303,-0.005136,0.001321,-0.008173,...,4.407246e-11,2.848324e-16,5.920432e-17,2.408456e-17,4.793126e-18,-1.151978e-19,-1.282902e-19,1.970825e-19,-1.397036e-19,1.327540e-20
2,0.774953,-0.591587,-0.038958,-0.155266,-0.009164,0.009816,-0.026717,-0.012116,0.001005,-0.036095,...,2.983274e-10,2.848324e-16,5.920432e-17,2.408456e-17,4.793126e-18,-1.151978e-19,-1.282902e-19,1.970825e-19,-1.397036e-19,1.327540e-20
3,0.788204,-0.581595,-0.107574,0.244563,-0.006063,0.007709,0.002228,0.000208,0.002261,-0.002722,...,2.505321e-08,3.545948e-15,-1.421295e-15,-1.352184e-15,-1.525480e-16,5.791804e-18,1.506590e-17,-1.006654e-17,-1.479364e-17,2.905970e-18
4,-0.581564,-0.188555,-0.012106,-0.098192,-0.009477,-0.000798,-0.015891,-0.026630,-0.006706,-0.049572,...,2.795621e-10,2.848324e-16,5.920432e-17,2.408456e-17,4.793126e-18,-1.151978e-19,-1.282902e-19,1.970825e-19,-1.397036e-19,1.327540e-20


In [21]:
explained_variance = pca.explained_variance_ratio_
np.sum(explained_variance[:400])

0.8357175108615409

In [22]:
y = df_final['Survived']
x = x_pca.iloc[:, 0:400]

df_final2 = pd.concat([x, y], axis = 1)

## Classification

In [23]:
# 5-Fold Cross Validation
def cross_validation (df, func):
    from sklearn.model_selection import KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 10)
    kf_accuracy = []
    
    for train, test in kf.split(df):
        X_train = df.iloc[train].loc[:, df.columns != 'Survived']
        X_train = X_train.squeeze()
        X_test = df.iloc[test].loc[:, df.columns != 'Survived']
        y_train = df.iloc[train].loc[:,'Survived']
        y_test = df.iloc[test].loc[:,'Survived']
        
        reg = func.fit(X_train, y_train)
        y_hat = reg.predict(X_test)
        
        from sklearn.metrics import accuracy_score
        kf_accuracy.append(accuracy_score(y_test, y_hat))
                
    kf_ACCURACY = (1/5) * np.sum(kf_accuracy)
        
    return (kf_ACCURACY)

In [24]:
def conf_mat(y, y_hat):
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y, y_hat)
    
    return(cm)

### Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model = LogisticRegression(random_state=0)
model.fit(x,y)
y_hat = model.predict(x)

from sklearn.metrics import accuracy_score
model_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model_cv_accuracy = cross_validation(df_final2, LogisticRegression())

print('train accuracy = ', model_train_accuracy)
print('5 CV accuracy = ', model_cv_accuracy)

train accuracy =  0.8428731762065096
5 CV accuracy =  0.8103132257862031


In [26]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[501  48]
 [ 92 250]]

True Positives(TP) =  501

True Negatives(TN) =  250

False Positives(FP) =  48

False Negatives(FN) =  92


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.84      0.91      0.88       549
           1       0.84      0.73      0.78       342

    accuracy                           0.84       891
   macro avg       0.84      0.82      0.83       891
weighted avg       0.84      0.84      0.84       891



In [28]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.8218


In [29]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8684398194858318


### KNN Classifier

In [30]:
#knn classifier
from sklearn.neighbors import KNeighborsClassifier

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

mse2 = []

#verify through CV which neighbors result in minimum rmse
for i in range(1,50,1):
    mse2.append(cross_validation(df_final2,KNeighborsClassifier(n_neighbors=i)))


In [31]:
#find the n with least cv-rmse
rmse_knn = pd.DataFrame(zip(np.linspace(1,len(mse2),len(mse2)), mse2), columns = ['n','rmse'])
rmse_knn['rmse'].min()
rmse_knn.loc[rmse_knn['rmse'] == rmse_knn['rmse'].min()]
#n_neighbors = 2 produced the smallest mse

,n,rmse
1,2.0,0.760919


In [32]:
#run knn again with the best n
from sklearn.neighbors import KNeighborsClassifier

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model12 = KNeighborsClassifier(n_neighbors = 2)
model12.fit(x,y)

y_hat = model12.predict(x)

from sklearn.metrics import accuracy_score
model12_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model12_cv_accuracy = cross_validation(df_final2, KNeighborsClassifier(n_neighbors = 2))

print('train accuracy = ', model12_train_accuracy)
print('5 CV accuracy = ', model12_cv_accuracy)

train accuracy =  0.8911335578002245
5 CV accuracy =  0.7609189630280584


In [33]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[549   0]
 [ 97 245]]

True Positives(TP) =  549

True Negatives(TN) =  245

False Positives(FP) =  0

False Negatives(FN) =  97


In [34]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       549
           1       1.00      0.72      0.83       342

    accuracy                           0.89       891
   macro avg       0.92      0.86      0.88       891
weighted avg       0.91      0.89      0.89       891



In [35]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.8582


In [36]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model12, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8171611039890447


### LDA

In [37]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model2 = LinearDiscriminantAnalysis()
model2.fit(x,y)
y_hat = model2.predict(x)

from sklearn.metrics import accuracy_score
model2_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model2_cv_accuracy = cross_validation(df_final2, LinearDiscriminantAnalysis())

print('train accuracy = ', model2_train_accuracy)
print('5 CV accuracy = ', model2_cv_accuracy)

train accuracy =  0.9528619528619529
5 CV accuracy =  0.6948088632226477


In [38]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[530  19]
 [ 23 319]]

True Positives(TP) =  530

True Negatives(TN) =  319

False Positives(FP) =  19

False Negatives(FN) =  23


In [39]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96       549
           1       0.94      0.93      0.94       342

    accuracy                           0.95       891
   macro avg       0.95      0.95      0.95       891
weighted avg       0.95      0.95      0.95       891



In [40]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.9491


In [41]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model2, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.7562941482209884


### QDA

In [42]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model3 = QuadraticDiscriminantAnalysis()
model3.fit(x,y)
y_hat = model3.predict(x)

from sklearn.metrics import accuracy_score
model3_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model3_cv_accuracy = cross_validation(df_final2, QuadraticDiscriminantAnalysis())

print('train accuracy = ', model3_train_accuracy)
print('5 CV accuracy = ', model3_cv_accuracy)

train accuracy =  0.9393939393939394
5 CV accuracy =  0.6767497332245308


In [43]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[495  54]
 [  0 342]]

True Positives(TP) =  495

True Negatives(TN) =  342

False Positives(FP) =  54

False Negatives(FN) =  0


In [44]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95       549
           1       0.86      1.00      0.93       342

    accuracy                           0.94       891
   macro avg       0.93      0.95      0.94       891
weighted avg       0.95      0.94      0.94       891



In [45]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.9508


In [46]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model3, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.6850523756298763


### SVC

In [47]:
from sklearn.svm import SVC

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model4 = SVC(gamma='auto')
model4.fit(x,y)
y_hat = model4.predict(x)

from sklearn.metrics import accuracy_score
model4_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model4_cv_accuracy = cross_validation(df_final2, SVC(gamma='auto'))

print('train accuracy = ', model4_train_accuracy)
print('5 CV accuracy = ', model4_cv_accuracy)

train accuracy =  0.7867564534231201
5 CV accuracy =  0.7867177201682255


In [48]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[468  81]
 [109 233]]

True Positives(TP) =  468

True Negatives(TN) =  233

False Positives(FP) =  81

False Negatives(FN) =  109


In [49]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83       549
           1       0.74      0.68      0.71       342

    accuracy                           0.79       891
   macro avg       0.78      0.77      0.77       891
weighted avg       0.78      0.79      0.78       891



In [50]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.7669


In [51]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model4, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8518861803705419


### SVM

In [52]:
#kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’

from sklearn.svm import SVC

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model4 = SVC(gamma='auto', kernel = 'linear')
model4.fit(x,y)
y_hat = model4.predict(x)

from sklearn.metrics import accuracy_score
model4_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model4_cv_accuracy = cross_validation(df_final2,SVC(gamma='auto', kernel = 'linear'))

print('train accuracy = ', model4_train_accuracy)
print('5 CV accuracy = ', model4_cv_accuracy)

train accuracy =  0.8765432098765432
5 CV accuracy =  0.8215115184232001


In [53]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[512  37]
 [ 73 269]]

True Positives(TP) =  512

True Negatives(TN) =  269

False Positives(FP) =  37

False Negatives(FN) =  73


In [54]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90       549
           1       0.88      0.79      0.83       342

    accuracy                           0.88       891
   macro avg       0.88      0.86      0.87       891
weighted avg       0.88      0.88      0.88       891



In [55]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.8596


In [56]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model4, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8697628422946944


In [57]:
#kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’

from sklearn.svm import SVC

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model4 = SVC(gamma='auto', kernel = 'poly')
model4.fit(x,y)
y_hat = model4.predict(x)

from sklearn.metrics import accuracy_score
model4_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model4_cv_accuracy = cross_validation(df_final2,SVC(gamma='auto', kernel = 'poly'))

print('train accuracy = ', model4_train_accuracy)
print('5 CV accuracy = ', model4_cv_accuracy)

train accuracy =  0.6161616161616161
5 CV accuracy =  0.6161195154102066


In [58]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[549   0]
 [342   0]]

True Positives(TP) =  549

True Negatives(TN) =  0

False Positives(FP) =  0

False Negatives(FN) =  342


In [59]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.62      1.00      0.76       549
           1       0.00      0.00      0.00       342

    accuracy                           0.62       891
   macro avg       0.31      0.50      0.38       891
weighted avg       0.38      0.62      0.47       891



In [60]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.5000


In [61]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model4, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8457646717533154


In [62]:
#kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’

from sklearn.svm import SVC

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model4 = SVC(gamma='auto', kernel = 'rbf')
model4.fit(x,y)
y_hat = model4.predict(x)

from sklearn.metrics import accuracy_score
model4_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model4_cv_accuracy = cross_validation(df_final2,SVC(gamma='auto', kernel = 'rbf'))

print('train accuracy = ', model4_train_accuracy)
print('5 CV accuracy = ', model4_cv_accuracy)

train accuracy =  0.7867564534231201
5 CV accuracy =  0.7867177201682255


In [63]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[468  81]
 [109 233]]

True Positives(TP) =  468

True Negatives(TN) =  233

False Positives(FP) =  81

False Negatives(FN) =  109


In [64]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83       549
           1       0.74      0.68      0.71       342

    accuracy                           0.79       891
   macro avg       0.78      0.77      0.77       891
weighted avg       0.78      0.79      0.78       891



In [65]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.7669


In [66]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model4, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8518861803705419


In [67]:
#kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’

from sklearn.svm import SVC

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model4 = SVC(gamma='auto', kernel = 'sigmoid')
model4.fit(x,y)
y_hat = model4.predict(x)

from sklearn.metrics import accuracy_score
model4_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model4_cv_accuracy = cross_validation(df_final2,SVC(gamma='auto', kernel = 'sigmoid'))

print('train accuracy = ', model4_train_accuracy)
print('5 CV accuracy = ', model4_cv_accuracy)

train accuracy =  0.6161616161616161
5 CV accuracy =  0.6161195154102066


In [68]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[549   0]
 [342   0]]

True Positives(TP) =  549

True Negatives(TN) =  0

False Positives(FP) =  0

False Negatives(FN) =  342


In [69]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.62      1.00      0.76       549
           1       0.00      0.00      0.00       342

    accuracy                           0.62       891
   macro avg       0.31      0.50      0.38       891
weighted avg       0.38      0.62      0.47       891



In [70]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.5000


In [71]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model4, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8524138224024425


### Decision Tree

In [72]:
from sklearn.tree import DecisionTreeClassifier

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

acc2 = []

for i in range(1,20,1):
    acc2.append(cross_validation(df_final,DecisionTreeClassifier(max_depth=i)))

In [73]:
acc_tree = pd.DataFrame(zip(np.linspace(1,len(acc2),len(acc2)), acc2), columns = ['n','accuracy'])
acc_tree.loc[acc_tree['accuracy'] == acc_tree['accuracy'].max()]
#max_depth = 19 produced the largest accuraccy

,n,accuracy
16,17.0,0.828247


In [74]:
from sklearn.tree import DecisionTreeClassifier

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model5 = DecisionTreeClassifier(max_depth=17)
model5.fit(x,y)

y_hat = model5.predict(x)


from sklearn.metrics import accuracy_score
model5_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model5_cv_accuracy = cross_validation(df_final2,DecisionTreeClassifier(max_depth=17))

print('train accuracy = ', model5_train_accuracy)
print('5 CV accuracy = ', model5_cv_accuracy)

train accuracy =  0.9854096520763187
5 CV accuracy =  0.7519553072625699


In [75]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[549   0]
 [ 13 329]]

True Positives(TP) =  549

True Negatives(TN) =  329

False Positives(FP) =  0

False Negatives(FN) =  13


In [76]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       549
           1       1.00      0.96      0.98       342

    accuracy                           0.99       891
   macro avg       0.99      0.98      0.98       891
weighted avg       0.99      0.99      0.99       891



In [77]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.9810


In [78]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model5, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.7244286603570609


### Decision Tree Pruning

In [79]:
from sklearn.tree import DecisionTreeClassifier

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

acc2 = []

alpha_values = np.linspace(0.0, 0.2, 200)

for i in range(len(alpha_values)-1):
    acc2.append(cross_validation(df_final,DecisionTreeClassifier(ccp_alpha = i)))

In [80]:
acc_prune_tree = pd.DataFrame(zip(alpha_values, acc2), columns = ['n','accuracy'])
acc_prune_tree.loc[acc_prune_tree['accuracy'] == acc_prune_tree['accuracy'].max()]
#max_depth = 19 produced the largest accuraccy

,n,accuracy
0,0.0,0.821505


In [81]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

model6 = DecisionTreeClassifier(ccp_alpha = 0)
#plot_tree(model6, filled=False)
#plt.show()
model6.fit(x,y)

y_hat = model6.predict(x)


from sklearn.metrics import accuracy_score
model6_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model6_cv_accuracy = cross_validation(df_final2,DecisionTreeClassifier(ccp_alpha = 0))

print('train accuracy = ', model6_train_accuracy)
print('5 CV accuracy = ', model6_cv_accuracy)


train accuracy =  0.9977553310886644
5 CV accuracy =  0.7508442658966794


In [82]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[549   0]
 [  2 340]]

True Positives(TP) =  549

True Negatives(TN) =  340

False Positives(FP) =  0

False Negatives(FN) =  2


In [83]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       549
           1       1.00      0.99      1.00       342

    accuracy                           1.00       891
   macro avg       1.00      1.00      1.00       891
weighted avg       1.00      1.00      1.00       891



In [84]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.9971


In [85]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model6, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.7428390311050627


### Random Forest

In [86]:
# random forest
from sklearn.ensemble import RandomForestClassifier

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

#m = sqrt(p)+1 features
nfeatures = df_final2.shape[1] - 1
model7 = RandomForestClassifier(max_features = int(np.sqrt(nfeatures))+1, random_state = 1) #random_state ensure random bagging
model7.fit(x,y)

y_hat = model7.predict(x)

from sklearn.metrics import accuracy_score
model7_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model7_cv_accuracy = cross_validation(df_final2,RandomForestClassifier(max_features = int(np.sqrt(nfeatures))+1, random_state = 1))

print('train accuracy = ', model7_train_accuracy)
print('5 CV accuracy = ', model7_cv_accuracy)

train accuracy =  0.9977553310886644
5 CV accuracy =  0.8114179900822297


In [87]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[549   0]
 [  2 340]]

True Positives(TP) =  549

True Negatives(TN) =  340

False Positives(FP) =  0

False Negatives(FN) =  2


In [88]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       549
           1       1.00      0.99      1.00       342

    accuracy                           1.00       891
   macro avg       1.00      1.00      1.00       891
weighted avg       1.00      1.00      1.00       891



In [89]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.9971


In [90]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model7, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8596343784640084


### Bagging

In [91]:
# Bagging
from sklearn.ensemble import RandomForestClassifier

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

#m = p features
nfeatures = df_final2.shape[1] - 1
model8 = RandomForestClassifier(max_features = nfeatures, random_state = 1) #random_state ensure random bagging
model8.fit(x,y)

y_hat = model8.predict(x)

from sklearn.metrics import accuracy_score
model8_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model8_cv_accuracy = cross_validation(df_final2,RandomForestClassifier(max_features = nfeatures, random_state = 1))

print('train accuracy = ', model8_train_accuracy)
print('5 CV accuracy = ', model8_cv_accuracy)

train accuracy =  0.9977553310886644
5 CV accuracy =  0.8091456907915385


In [92]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[547   2]
 [  0 342]]

True Positives(TP) =  547

True Negatives(TN) =  342

False Positives(FP) =  2

False Negatives(FN) =  0


In [93]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       549
           1       0.99      1.00      1.00       342

    accuracy                           1.00       891
   macro avg       1.00      1.00      1.00       891
weighted avg       1.00      1.00      1.00       891



In [94]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.9982


In [95]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model8, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8611448372364866


### Boosting

In [96]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

x = df_final2.loc[:, df_final2.columns != 'Survived']
y = df_final2['Survived']

NTREES = 5000
INTERACTION_DEPTH = 4
SHRINKAGE_PARAM = 0.2 
model9 = GradientBoostingClassifier(\
                        n_estimators = NTREES,\
                        learning_rate = SHRINKAGE_PARAM,\
                        max_depth = INTERACTION_DEPTH,\
                        random_state = 1)

model9.fit(x, y)

y_hat = model9.predict(x)

from sklearn.metrics import accuracy_score
model9_train_accuracy = accuracy_score(y, y_hat)
#score = model.score(x, y)

model9_cv_accuracy = cross_validation(df_final2,GradientBoostingClassifier(\
                        n_estimators = NTREES,\
                        learning_rate = SHRINKAGE_PARAM,\
                        max_depth = INTERACTION_DEPTH,\
                        random_state = 1))

print('train accuracy = ', model9_train_accuracy)
print('5 CV accuracy = ', model9_cv_accuracy)

train accuracy =  0.9977553310886644
5 CV accuracy =  0.80585022911305


In [97]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[549   0]
 [  2 340]]

True Positives(TP) =  549

True Negatives(TN) =  340

False Positives(FP) =  0

False Negatives(FN) =  2


In [98]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       549
           1       1.00      0.99      1.00       342

    accuracy                           1.00       891
   macro avg       1.00      1.00      1.00       891
weighted avg       1.00      1.00      1.00       891



In [99]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.9971


In [100]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model9, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.8559997568306068


### Bayes Method

In [101]:
#df_final = pd.concat([df_cont2, df_cat2, df_response], axis = 1)
df_final

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Sex_nan,Ticket_110152,Ticket_110413,...,Ticket_W./C. 6609,Ticket_W.E.P. 5734,Ticket_W/C 14208,Ticket_WE/P 5735,Ticket_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Survived
0,-0.579151,-0.160673,-0.034249,0.0,0.006766,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-0.193050,-0.277527,-0.034249,0.0,0.028929,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,-0.579151,-0.189887,-0.000000,0.0,0.007396,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,-0.193050,-0.255617,-0.034249,0.0,0.049553,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,-0.579151,-0.255617,-0.000000,0.0,0.007512,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-0.386101,-0.197190,-0.000000,0.0,0.012132,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
887,-0.193050,-0.138763,-0.000000,0.0,0.027996,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
888,-0.579151,-0.204493,-0.034249,0.0,0.021884,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
889,-0.193050,-0.189887,-0.000000,0.0,0.027996,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [102]:
#df_response.value_counts() # need this for priors

#Gaussian
from sklearn.naive_bayes import GaussianNB

x1 = df_cont2
y = df_response

model10 = GaussianNB()
model10.fit(x1,y)

y_hat1 = model10.predict(x1)


#Multinomial (for discrete values)
from sklearn.naive_bayes import MultinomialNB

x2 = df_cat2
y = df_response

model11 = MultinomialNB()
model11.fit(x2,y)

y_hat2 = model11.predict(x2)

pi_0 = 549/891
pi_1 = 342/891

y_hat = np.floor(y_hat1*y_hat2 * (pi_0/pi_1))

from sklearn.metrics import accuracy_score
model10_train_accuracy = accuracy_score(y, y_hat)

print('train accuracy = ', model10_train_accuracy)

train accuracy =  0.755331088664422


In [103]:
cm = conf_mat(y, y_hat)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[546   3]
 [215 127]]

True Positives(TP) =  546

True Negatives(TN) =  127

False Positives(FP) =  3

False Negatives(FN) =  215


In [104]:
from sklearn.metrics import classification_report
print(classification_report(y, y_hat))

              precision    recall  f1-score   support

           0       0.72      0.99      0.83       549
           1       0.98      0.37      0.54       342

    accuracy                           0.76       891
   macro avg       0.85      0.68      0.69       891
weighted avg       0.82      0.76      0.72       891



In [105]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y, y_hat)
print('ROC AUC : {:.4f}'.format(ROC_AUC))

ROC AUC : 0.6829


In [106]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model10, x, y, cv = 5, scoring='roc_auc')
scores = (1/5)*np.sum(scores)
print('Cross-validation scores:{}'.format(scores))

Cross-validation scores:0.700811768047684
